In [62]:
import pandas as pd
import numpy as np
from sklearn.metrics import accuracy_score, mean_squared_error, confusion_matrix, cohen_kappa_score
from sklearn.model_selection import RandomizedSearchCV
import matplotlib.pyplot as plt

import xgboost as xgb

from cleaning import clean_data

In [55]:
X_train, Y_train, X_test, Y_test = clean_data('./petfinder-adoption-prediction/train/train.csv')

In [56]:
train = xgb.DMatrix(X_train, label = Y_train)
test = xgb.DMatrix(X_test, label = Y_test)

In [57]:
'''params = {
    'max_depth' : 6,
    'eta' : 0.3,
    'objective' : 'multi:softmax',
    'num_class':5
}
epochs = 25
model = xgb.train(params, train, epochs)
Y_pred = model.predict(test)'''

"params = {\n    'max_depth' : 6,\n    'eta' : 0.3,\n    'objective' : 'multi:softmax',\n    'num_class':5\n}\nepochs = 25\nmodel = xgb.train(params, train, epochs)\nY_pred = model.predict(test)"

In [58]:
'''hyper_params = {
    'learning_rate': np.arange(0.01, 0.5, 0.05),
    'max_depth':range(3,10),
    'colsample_bytree' : np.arange(0.5, 1, 0.05),
    'objective': ['multi:softmax'],
    'num_class':[5],
}

model = xgb.XGBClassifier()
search = RandomizedSearchCV(model, param_distributions=hyper_params, n_iter=30, cv=3)
search.fit(X_train, Y_train)'''

In [69]:
'''model = xgb.XGBClassifier(
    learning_rate = 0.1,
    n_estimators = 1000,
    max_depth = 5,
    min_child_weight = 1,
    gamma = 0,
    subsample = 0.8,
    colsample_bytree = 0.8,
    objective = 'multi:softmax',
    num_class = 5,
)
xgb_param = model.get_xgb_params()
cvresult = xgb.cv(xgb_param, train, num_boost_round=model.get_params()['n_estimators'], nfold=5,
            metrics='auc', early_stopping_rounds=50)
model.set_params(n_estimators=cvresult.shape[0])

#model.fit(X_train, Y_train)
#Y_pred = model.predict(X_test)'''

XGBClassifier(base_score=None, booster=None, callbacks=None,
              colsample_bylevel=None, colsample_bynode=None,
              colsample_bytree=0.8, early_stopping_rounds=None,
              enable_categorical=False, eval_metric=None, gamma=0, gpu_id=None,
              grow_policy=None, importance_type=None,
              interaction_constraints=None, learning_rate=0.1, max_bin=None,
              max_cat_to_onehot=None, max_delta_step=None, max_depth=5,
              max_leaves=None, min_child_weight=1, missing=nan,
              monotone_constraints=None, n_estimators=141, n_jobs=None,
              num_class=5, num_parallel_tree=None, objective='multi:softmax',
              predictor=None, random_state=None, ...)

In [84]:
'''hyper_params = {'max_depth':range(1,10),
                'min_child_weight':range(1,6)}
search = RandomizedSearchCV(estimator=xgb.XGBClassifier(learning_rate = 0.1, n_estimators=141,
                                                        gamma=0, subsample=0.8, colsample_bytree=0.8,objective= 'multi:softmax'), 
                            param_distributions = hyper_params, verbose = 1, cv = 5)
search.fit(X_train, Y_train)
print(search.best_score_)
print(search.best_estimator_)'''

Fitting 5 folds for each of 10 candidates, totalling 50 fits
0.40666984278227725
XGBClassifier(base_score=0.5, booster='gbtree', callbacks=None,
              colsample_bylevel=1, colsample_bynode=1, colsample_bytree=0.8,
              early_stopping_rounds=None, enable_categorical=False,
              eval_metric=None, gamma=0, gpu_id=-1, grow_policy='depthwise',
              importance_type=None, interaction_constraints='',
              learning_rate=0.1, max_bin=256, max_cat_to_onehot=4,
              max_delta_step=0, max_depth=5, max_leaves=0, min_child_weight=3,
              missing=nan, monotone_constraints='()', n_estimators=141,
              n_jobs=0, num_parallel_tree=1, objective='multi:softmax',
              predictor='auto', random_state=0, reg_alpha=0, ...)


In [ ]:
hyper_params = {'gamma':[i/10.0 for i in range(0,5)]}
search = RandomizedSearchCV(estimator=xgb.XGBClassifier(learning_rate = 0.1, n_estimators=141, max_depth=5, min_child_weight=3, 
                                                        subsample=0.8, colsample_bytree=0.8,objective= 'multi:softmax'), 
                            param_distributions = hyper_params, verbose = 1, cv = 5)
search.fit(X_train, Y_train)
print(search.best_score_)
print(search.best_estimator_)

In [85]:
model = xgb.XGBClassifier(learning_rate = 0.1, n_estimators=141, max_depth=5,
                            min_child_weight=3, gamma=0, subsample=0.8, colsample_bytree=0.8,objective= 'multi:softmax')
model.fit(X_train, Y_train)
Y_pred = model.predict(X_test)

In [86]:
print('Confusion Matrix\n',confusion_matrix(Y_test, Y_pred))
print('Cohen Kappa', cohen_kappa_score(Y_test, Y_pred, weights='quadratic'))
print('Accuracy Score', accuracy_score(Y_test, Y_pred))
print('MSE', mean_squared_error(Y_test, Y_pred))

Confusion Matrix
 [[  0  42  35   7  43]
 [  0 268 329  76 234]
 [  2 213 471 172 389]
 [  1 129 273 183 373]
 [  1  97 214  71 875]]
Cohen Kappa 0.3196618557002461
Accuracy Score 0.39951089373054693
MSE 1.9137394397510004
